In [1]:
# Import the necessary packages

In [2]:
import nltk
import numpy as np
import random
import string
import warnings
warnings.filterwarnings('ignore')
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout,Embedding,LSTM,GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
import re

In [3]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [4]:
# Load the json file 

In [5]:
with open('intents.json') as file:
    data=json.load(file)
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'],
   'responses': ['Hello', 'Hi', 'Hi there']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Thanks for the help'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    "You're most welcome!"]},
  {'tag': 'about',
   'patterns': ['Who are you?', 'What are you?', 'Who you are?'],
   'responses': ['I.m Joana, your bot assistant',
    "I'm Joana, an Artificial Intelligent bot"]},
  {'tag': 'name',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Joana.',
    "I'm Joana!",
    'Just call me as Joana']},
  {'tag': 'time',
   'patterns': ['what is the best time to visit your office',
    'Your office worki

In [6]:
trainining_sent=[]
training_labels=[]
labels=[]
responses=[]

In [7]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        trainining_sent.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
num_classes=len(labels)
num_classes    

11

In [8]:
print(trainining_sent,end=" ")

['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay', 'Bye', 'See you later', 'Goodbye', 'Thanks', 'Thank you', "That's helpful", 'Thanks for the help', 'Who are you?', 'What are you?', 'Who you are?', 'what is your name', 'what should I call you', 'whats your name?', 'what is the best time to visit your office', 'Your office working time', 'is it open your office in the afternoon', 'Could you help me?', 'give me a hand please', 'Can you help?', 'What can you do for me?', 'I need a support', 'I need a help', 'support me please', 'How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered', 'I need to create a new account', 'how to open a new account', 'I want to create an account', 'can you create an account for me', 'how to open a new account', 'have a complaint', 'I want to raise a complaint', 'there is a complaint about a service'] 

In [9]:
print(training_labels,end="")

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'about', 'about', 'about', 'name', 'name', 'name', 'time', 'time', 'time', 'help', 'help', 'help', 'help', 'help', 'help', 'help', 'options', 'options', 'options', 'options', 'options', 'createaccount', 'createaccount', 'createaccount', 'createaccount', 'createaccount', 'complaint', 'complaint', 'complaint']

In [10]:
print(responses,end='')

[['Hello', 'Hi', 'Hi there'], ['See you later', 'Have a nice day', 'Bye! Come back again'], ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"], ['I.m Joana, your bot assistant', "I'm Joana, an Artificial Intelligent bot"], ['You can call me Joana.', "I'm Joana!", 'Just call me as Joana'], ['You can visit our office from Monday to Saturday at any time between 9.00 AM to 5.30 PM'], ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], ['Tell me how can assist you', 'Tell me your problem to assist you', 'Yes Sure, How can I support you'], ['I can help you to solve your problems related to our services'], ['You can just easily create a new account from our web site', 'Just go to our web site and follow the guidelines to create a new account'], ['Please provide us your complaint in order to assist you', 'Please mention your complaint, we will reach you and sorry for any inconvenience caused']]

In [11]:
from nltk.corpus import stopwords
st=stopwords.words('english')

In [12]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [13]:
# Convert text data into vector format

In [14]:
vocab_size = 1000
embedding_dim = 16     
max_len = 20
oov_token = "<OOV>"   


tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(trainining_sent)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(trainining_sent)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [15]:
from sklearn.preprocessing import LabelEncoder
enc=LabelEncoder()
enc.fit(training_labels)
training_labels=enc.transform(training_labels)

In [16]:
# Model 

In [17]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 11)                187       
Total params: 16,731
Trainable params: 16,731
Non-trainable params: 0
_________________________________________________________________


In [18]:
his=model.fit(padded_sequences,np.array(training_labels),epochs=750)

Epoch 1/750
2/2 [==============================] - 2s 8ms/step - loss: 2.3986 - accuracy: 0.0696
Epoch 2/750
2/2 [==============================] - 0s 5ms/step - loss: 2.3977 - accuracy: 0.1230
Epoch 3/750
2/2 [==============================] - 0s 4ms/step - loss: 2.3960 - accuracy: 0.1230
Epoch 4/750
2/2 [==============================] - 0s 5ms/step - loss: 2.3946 - accuracy: 0.1230
Epoch 5/750
2/2 [==============================] - 0s 4ms/step - loss: 2.3934 - accuracy: 0.1230
Epoch 6/750
2/2 [==============================] - 0s 4ms/step - loss: 2.3924 - accuracy: 0.1126
Epoch 7/750
2/2 [==============================] - 0s 5ms/step - loss: 2.3910 - accuracy: 0.1334
Epoch 8/750
2/2 [==============================] - 0s 4ms/step - loss: 2.3910 - accuracy: 0.1021
Epoch 9/750
2/2 [==============================] - 0s 5ms/step - loss: 2.3897 - accuracy: 0.1126
Epoch 10/750
2/2 [==============================] - 0s 4ms/step - loss: 2.3891 - accuracy: 0.1126
Epoch 11/750
2/2 [===========

In [21]:
model.save('chat_model')

INFO:tensorflow:Assets written to: chat_model\assets


In [22]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle)
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(enc, ecn_file)

In [20]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder



import random
import pickle


with open("intents_.json") as file:
    data = json.load(file)




def chat():
   
    model = keras.models.load_model('chat_model')


    
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)


  
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)


    
    max_len = 20
    
    while True:
        print("User:" , end="")
        inp = input()
        if inp.lower() == "quit":
            break


        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])


        for i in data['intents']:
            if i['tag'] == tag:
                print( "ChatBot:"  , np.random.choice(i['responses']))




print("Start messaging with the bot (type quit to stop)!")
chat()


Start messaging with the bot (type quit to stop)!
User:

 hai


ChatBot: You're most welcome!
User:

 what is your name


ChatBot: Just call me as Joana
User:

...

 have problem


ChatBot: Hi
User:

 thanks


ChatBot: You're most welcome!
User:

 quit
